# 先引用套件 再設一個公式 起始值白色 這樣之後for loop就可以帶入

In [ ]:
import folium
import requests
import re
import os
from time import gmtime,strftime,localtime
import json

In [ ]:
def color_producer(values):
    color="white"
    if values<=10 :
        color = 'green'
    elif values>10 and values<=15:
        color = 'yellow'
    elif values>15 and values<=20:
        color= "red"
    else:
        color= "purple"
    cr=  0.7
    return [color,cr]

# 可以從json看到除了有 pm2.5_avg 縣市 還有 座標這樣就不用合併

In [ ]:
url ='https://data.epa.gov.tw/api/v2/aqx_p_432?api_key=e8dd42e6-9b8b-43f8-991e-b3dee723a52d&limit=1000&sort=ImportDate%20desc&format=JSON'
html = requests.get(url)
json=json.loads(html.text)
json

In [ ]:
#  m我們一樣設定中心位置  至於tiles以下類型
# 「OpenStreetMap」
# 「Mapbox Bright」 (Limited levels of zoom for free tiles)
# 「Mapbox Control Room」 (Limited levels of zoom for free tiles)
# 「Stamen」 (Terrain, Toner, and Watercolor)
# 「Cloudmade」 (Must pass API key)
# 「Mapbox」 (Must pass API key)
# 「CartoDB」 (positron and dark_matter)
m = folium.Map(location=(23.5,121), zoom_start=8,tiles='Stamen Terrain',title="全台PM2.5分布圖 資料來源:環境資料開放平臺")
folium.map.Marker(location=(23.7,121.5),icon=folium.DivIcon(icon_size=(300,36),icon_anchor=(0,0),html="<div style='font-size:1.8em;color:red;'>全台PM2.5分布圖 <br>資料來源:環境資料開放平臺 <br> 0-9綠<br> 10-15黃<br>16-20紅<br> >20 紫 </div>")).add_to(m)
#順便再做一個正規劃 去除不要的字元 
isdigit = re.compile(r'^\d+\.*\d*$')
fg = folium.FeatureGroup(name="PM2.5")

#  開始用for loop將json下的records字典中資料一一提取
#  如果資料的座標且符合正規化 就進行以下的動作

In [ ]:
for j in json['records']: 
    pm225 = j["pm2.5_avg"]
    if j["latitude"]!= "" and j["longitude"]!="" and isdigit.match(pm225) is not None:
        values = int(pm225)
        cc = color_producer(values)
        loc = (j["latitude"],j["longitude"])
# 我將tooltip設定關鍵字好讓視覺化更清楚 當然滑鼠移動到小圈圈內也是有資料
        fg.add_child(folium.CircleMarker(location=loc,radius=10,tooltip=j['county']+j["sitename"]+'\n'+'PM2.5值:'+j['pm2.5_avg'],
            fill_color=cc[0],color = 'grey',fill_opacity=cc[1]))
m.add_child(fg)                    

# m.save('pm25.html')

# 最後 天氣那篇爬蟲因為沒有縣市座標 從這邊抓取存入

In [ ]:
import pandas as pd
data=[]
for j in json['records']: 
    county=j['county']
    lat=j['latitude']
    log=j["longitude"]
    data.append([county,lat,log])
    df=pd.read_excel('county.xlsx')